In [2]:
# pip install datasets transformers[sentencepiece]

In [3]:
# pip install tensorflow

In [4]:
from datasets import load_dataset

The \u0085 unicode character in some of the text files was causing the files to be split into multiple rows. So I need to remove this character from the text files.

In [13]:
import os
dirs = ['data/aclImdb/train/pos', 'data/aclImdb/train/neg', 'data/aclImdb/test/pos', 'data/aclImdb/test/neg']
for dir in dirs:
    for dir_file in os.listdir(dir):
        # rename or store the new file name for later rename
    # Read in the file
        with open(dir + '/' + dir_file, 'r') as file:
    #         length = len(file.readlines())
    #         if length > 1:
    #             print(length)
            filedata = file.read()
    #         if '\u0085' in filedata:
    #             print(dir_file + '\n' + filedata)

    # Replace the target string

        filedata = filedata.replace('\u0085', ' ')

    # Write the file out again
        with open(dir + '/' + dir_file, 'w') as file:
            file.write(filedata)

In [17]:

dirs = ['data/aclImdb/train/pos', 'data/aclImdb/train/neg', 'data/aclImdb/test/pos', 'data/aclImdb/test/neg']
for dir in dirs:
    for dir_file in os.listdir(dir):
            file_number, file_end = dir_file.split('_')
            num = file_number.zfill(5)  # num is 5 characters long with leading 0

            
            new_file = "{}_{}".format(num, file_end)
            os.rename(dir + '/' + dir_file, dir + '/' + new_file)


In [35]:
# Put the files into ordered lists, one for train and one for test
dirs = ['train','test']
files = {}
for dir in dirs:
    files[dir] = []
    subdirs = ['pos', 'neg']
    for subdir in subdirs:
        files_tmp = []
        for dir_file in os.listdir('data/aclImdb/' + dir + '/' + subdir):
            files_tmp += [dir_file]
        files_tmp = sorted(files_tmp)
        files[dir].extend(files_tmp)

In [51]:
# Get two lists of the ratings, taken from the filenames
dirs = ['train','test']
ratings = {}
for dir in dirs:
    ratings[dir] = []
    for file in files[dir]:
        rating = int(file.split('_')[1].split('.')[0])
        ratings[dir].append(rating)

In [244]:
ratings_pos = np.array(ratings['test'][:12])
ratings_neg = np.array(ratings['test'][12500:12512])
ratings_np = np.concatenate((ratings_pos, ratings_neg))

In [245]:
ratings_np

array([10, 10,  7,  7, 10,  7,  7,  9,  9,  7,  7,  8,  2,  3,  3,  4,  4,
        4,  3,  1,  2,  4,  3,  3])

In [71]:
import statistics
import numpy as np
mean = statistics.mean(ratings['train'])

In [75]:
tr_ratings = np.array(ratings['train'])
statistics.mean(abs(tr_ratings - mean))

3.26076

In [18]:
data = load_dataset("text", data_files={'train': ['data/aclImdb/train/pos/*.txt', 'data/aclImdb/train/neg/*.txt'], 'test': ['data/aclImdb/test/pos/*.txt', 'data/aclImdb/test/neg/*.txt']})

Resolving data files:   0%|          | 0/25000 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25000 [00:00<?, ?it/s]

Using custom data configuration default-2f5596483a59ffb3


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /home/mark/.cache/huggingface/datasets/text/default-2f5596483a59ffb3/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
small_data_rows = 128
small_data = load_dataset("text", data_files={'train': ['data/aclImdb/train/pos/*.txt', 'data/aclImdb/train/neg/*.txt'], 'test': ['data/aclImdb/test/pos/*.txt', 'data/aclImdb/test/neg/*.txt']})

In [20]:
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 25000
    })
})

In [54]:
data['train'] = data['train'].add_column("rating", ratings['train'])
data['test'] = data['test'].add_column("rating", ratings['test'])

In [14]:
from transformers import AutoTokenizer, DataCollatorWithPadding

2021-12-10 12:39:16.042114: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 12:39:16.042142: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [1]:
# pip install datasets[s3]

Note: you may need to restart the kernel to use updated packages.


In [5]:
import datasets
from datasets.filesystems import S3FileSystem
from datasets import load_from_disk

In [6]:
!mkdir -p ~/.aws
!cp ../aws_config ~/.aws/config

In [7]:
from botocore.session import Session

s3_session = Session()

In [8]:
s3 = datasets.filesystems.S3FileSystem(session=s3_session)

In [9]:
s3.ls('mlzoomcamp-capstone')

['mlzoomcamp-capstone/dataset_dict.json',
 'mlzoomcamp-capstone/test',
 'mlzoomcamp-capstone/train']

In [64]:
data.save_to_disk('s3://mlzoomcamp-capstone', fs=s3)

In [141]:
MAX_LENGTH = 256
MAX_ROWS = 24
BATCH_SIZE = 8

In [10]:
data = load_from_disk('s3://mlzoomcamp-capstone',fs=s3)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'rating'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'rating'],
        num_rows: 25000
    })
})

In [86]:
tokens = tokenizer("This is a test", truncation=True, padding=True, max_length=MAX_LENGTH)

In [11]:
def data_subset(data, max_rows):
    return data.filter(lambda e, i: (i in range(int(max_rows / 2)) or i in range(int(len(data['train']) / 2), int(len(data['train']) / 2) + int(max_rows / 2))), with_indices=True)

def tokenize_function(example, tokenizer, batched=True):
    return tokenizer(example["text"], truncation=True, padding=True, max_length=MAX_LENGTH)

In [12]:
smaller_data = data_subset(data, MAX_ROWS)

NameError: name 'MAX_ROWS' is not defined

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [17]:
params_dictionary = {'epochs': 2, 'lr': 0.0005, 'layer_dropout': 0, 'att_dropout': 0, 'max_length': 512, 'batch_size': 8, 'max_rows': 25000}

In [18]:
import tensorflow as tf

In [19]:
from transformers import TFDistilBertModel, DistilBertConfig

In [20]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True, max_length=params_dictionary['max_length'])

In [35]:
def prepare_data(params_dictionary):
    max_rows = params_dictionary['max_rows']
    max_length = params_dictionary['max_length']
    smaller_data = data.filter(lambda e, i: (i in range(int(max_rows / 2)) or i in range(12500, 12500 + int(max_rows / 2))), with_indices=True)
    tokenized_data = smaller_data.map(tokenize_function, batched=True)
    print(tokenized_data['test'])
    
    tf_train_dataset = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["rating"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=params_dictionary['batch_size'],
        )
    tf_val_dataset = tokenized_data["test"].to_tf_dataset(
        columns=["attention_mask", "input_ids"],
        label_cols=["rating"],
        shuffle=False,
        collate_fn=data_collator,
        batch_size=params_dictionary['batch_size'],
        )
    return tf_train_dataset, tf_val_dataset

In [31]:
tokenized_data['train']

NameError: name 'tokenized_data' is not defined

In [22]:
def config_model(params_dictionary):
 
    # Configure DistilBERT's initialization
    config = DistilBertConfig(dropout=params_dictionary['layer_dropout'] , 
                              attention_dropout=params_dictionary['att_dropout'], 
                              output_hidden_states=True)

    # The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
    # and without any specific head on top.
    distilBERT = TFDistilBertModel.from_pretrained(checkpoint, config=config)

    # Make DistilBERT layers untrainable
    for layer in distilBERT.layers:
        layer.trainable = False
    return distilBERT

In [23]:
def build_model(transformer, params_dictionary):
    """
    Template for building a model off of the BERT or DistilBERT architecture
    for a binary classification task.
    
    Input:
      - transformer:  a base Hugging Face transformer model object (BERT or DistilBERT)
                      with no added classification head attached.
      - max_length:   integer controlling the maximum number of encoded tokens 
                      in a given sequence.
    
    Output:
      - model:        a compiled tf.keras.Model with added classification layers 
                      on top of the base pre-trained model architecture.
    """
    
    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE) 
    
    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(params_dictionary['max_length'],), 
                                            name='input_ids', 
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(params_dictionary['max_length'],), 
                                                  name='attention_mask', 
                                                  dtype='int32')
    
    # DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tf.Tensor of shape (batch_size, sequence_length, hidden_size=768).
    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
    
    # We only care about DistilBERT's output for the [CLS] token, 
    # which is located at index 0 of every encoded sequence.  
    # Splicing out the [CLS] tokens gives us 2D data.
    # cls_token = last_hidden_state[:, 0, :]
    
    ##                                                 ##
    ## Define additional dropout and dense layers here ##
    ##                                            
    
    # vector = tf.reshape(last_hidden_state, [-1])

    vector = tf.keras.layers.GlobalAvgPool1D()(last_hidden_state)

    output1 = tf.keras.layers.Dense(128, 
                                   activation='relu',
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(vector)
    
    output2 = tf.keras.layers.Dense(128, 
                                   activation='relu',
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(output1)
    
    output3 = tf.keras.layers.Dense(64, 
                                   activation='relu',
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(output2)

    output4 = tf.keras.layers.Dense(10, 
                                   activation='relu',
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(output3)
    
    # Define a single node that makes up the output layer (for binary classification)
    output5 = tf.keras.layers.Dense(1, 
                                   activation=None,
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(output4)
    
    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output5)
    
    # Compile the model
    model.compile(tf.keras.optimizers.Adam(learning_rate=params_dictionary['lr']), 
                  loss=MeanSquaredError(),
                  metrics=['mse'])
    
    return model

In [24]:
RANDOM_STATE = 1
from tensorflow.keras.losses import MeanSquaredError

In [25]:
distilBERT = config_model(params_dictionary)


2021-12-10 12:39:46.716578: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-10 12:39:46.716659: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-10 12:39:46.716705: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mark-laptop): /proc/driver/nvidia/version does not exist
2021-12-10 12:39:46.717070: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-10 12:39:46.739568: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, 

In [26]:
distilBERT_model = build_model(distilBERT, params_dictionary)

In [27]:
distilBERT_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 512, 768),                                                   
                                 hidden_states=((No                                           

In [28]:
distilBERT_model.load_weights('provisional_model.h5')

2021-12-10 12:40:01.742626: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.
2021-12-10 12:40:01.918641: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.


In [189]:
params_dictionary = {'epochs': 2, 'lr': 0.0005, 'layer_dropout': 0, 'att_dropout': 0, 'max_length': 512, 'batch_size': 8, 'max_rows': 24}

In [36]:
tf_train_dataset, tf_val_dataset = prepare_data(params_dictionary)

Loading cached processed dataset at /tmp/tmp9p8uibbe/mlzoomcamp-capstone/train/cache-25f4c7c6fd533119.arrow
Loading cached processed dataset at /tmp/tmp9p8uibbe/mlzoomcamp-capstone/test/cache-e8f5c4ab115d2066.arrow
Loading cached processed dataset at /tmp/tmp9p8uibbe/mlzoomcamp-capstone/train/cache-6164583956468d4b.arrow
Loading cached processed dataset at /tmp/tmp9p8uibbe/mlzoomcamp-capstone/test/cache-71db71ced1e00a8e.arrow


Dataset({
    features: ['attention_mask', 'input_ids', 'rating', 'text', 'token_type_ids'],
    num_rows: 25000
})


In [217]:
predictions = distilBERT_model.predict(tf_val_dataset)

In [30]:
tf_val_dataset

<PrefetchDataset shapes: ({attention_mask: (None, None), input_ids: (None, None)}, (None,)), types: ({attention_mask: tf.int64, input_ids: tf.int64}, tf.int64)>

In [224]:
np.array(predictions[:12]).mean()

7.2616544

In [225]:
np.array(predictions[12:]).mean()

4.2218375

In [226]:
predictions

array([[ 8.972178 ],
       [10.159443 ],
       [ 3.959504 ],
       [ 6.376056 ],
       [ 8.9968405],
       [ 5.8000817],
       [ 7.109173 ],
       [ 9.432082 ],
       [ 2.8598983],
       [ 8.451174 ],
       [ 7.4948816],
       [ 7.528542 ],
       [ 2.6925578],
       [ 4.1040683],
       [ 2.3468995],
       [ 8.318647 ],
       [ 2.6734893],
       [ 6.346175 ],
       [ 6.707441 ],
       [ 3.2532237],
       [ 3.3133276],
       [ 4.32001  ],
       [ 3.2285888],
       [ 3.3576155]], dtype=float32)

In [227]:
distilBERT_model.evaluate(tf_val_dataset)

3/3 [==============================] - 21s 5s/step - loss: 4.2710 - mse: 4.2710


[4.270998954772949, 4.270998954772949]

In [250]:
predictions_np = np.array(predictions)

In [261]:
diff = predictions_np[:,0] - ratings_np

In [262]:
diff

array([-1.02782154,  0.1594429 , -3.04049611, -0.62394381, -1.00315952,
       -1.19991827,  0.10917282,  0.43208218, -6.14010167,  1.45117378,
        0.49488163, -0.47145796,  0.69255781,  1.10406828, -0.65310049,
        4.31864738, -1.32651067,  2.34617519,  3.70744085,  2.25322366,
        1.31332755,  0.32001019,  0.22858882,  0.35761547])

In [266]:
sum(diff ** 2)/24

4.270999236775239

In [258]:
ratings_np

array([10, 10,  7,  7, 10,  7,  7,  9,  9,  7,  7,  8,  2,  3,  3,  4,  4,
        4,  3,  1,  2,  4,  3,  3])

In [267]:
import tflite_runtime.interpreter as tflite

ModuleNotFoundError: No module named 'tflite_runtime'